**Google Drive mount**
- for fasttext pretrained data

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 448 kB 40.9 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [28]:
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re
from PIL import Image
from io import BytesIO
from nltk.tokenize import RegexpTokenizer
import nltk
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer   
from konlpy.tag import Okt  

full_data = pd.read_excel('/content/gdrive/MyDrive/naver_restaurant.xlsx')

In [4]:
# device = torch.device('cuda')

NameError: ignored

In [5]:
full_data[:5]    

,store_name,store_type,star_point,star_point_qty,menu,blog_keyword,addr
0,안동장,중식당,4.23,945,"굴짬뽕/매운굴짬뽕/송이짬뽕/매운송이짬뽕,삼선짬뽕/매운 삼선짬뽕/삼선울면/삼선우동,짬...","안녕하다,누저,기입,니,이번,제,맛집,을지로,위치,오래되다,중식,안동장,이다,안동장...",서울특별시 중구 을지로3가 315-18번지
1,조선옥,"육류,고기요리",4.00,474,"양념소갈비(250g),갈비탕,대구탕,장국밥,냉면","날인,약속,있다,택시,타고,강남,옆,있다,지인,잠깐,을지로,를,가잔,아주,말,어차피...",서울특별시 중구 을지로3가 228번지
2,대림국수 을지로점,요리주점,4.37,456,"꼬치국수 온면(기본맛/순한맛)\n대표,모듬숯불꼬치 (8종)\n대표,꼬치국수 비빔면,...","을지로,볼일,있다,마침,저녁,시간,대림,상가,을지로,마땅하다,맛집,없다,대림,가로,...",서울특별시 중구 산림동 166-0번지
3,브라운치킨 동대문본점,"치킨,닭강정",4.29,100,"마늘치킨\n대표,후라이드치킨,양념치킨,간장치킨,웰빙파치킨,골뱅이무침+소면,치킨떡볶이...","하나,동대문,저,놀이터,정말,시즌,쇼핑,곳,젠,멀다,동대문,격변,동네,건가,불과,해...",서울특별시 중구 을지로6가 18-52번지
4,삼보호프,"맥주,호프",4.23,59,"노가리,마른안주,황도,번데기,야채","을지로,치킨,냄새,노상,맥주,바삭바삭하다,강냉이,치킨,바삭,옛날,치킨,끝내주다,맛있...",서울특별시 중구 주교동 196-0번지


In [6]:
word_list =[]
for i in range(0, len(full_data)):
    word1 = full_data.iloc[i][1].replace('/', ',').replace(' ', ',').split(',')
    if type(full_data.iloc[i][4]) != str:
        word2 = []
    else:        
        word2 = full_data.iloc[i][4].replace('/', ',').replace(' ', ',').split(',')
    word3 =  full_data.iloc[i][5].split(',')  
    
    word = word1 + word2 + word3
    
    word_list.append(word)


In [46]:
model = Word2Vec.load('/content/gdrive/MyDrive/ko.bin')
model.wv.save_word2vec_format('/content/gdrive/MyDrive/ko.bin.gz', binary=False)

In [47]:

model_2 = Word2Vec(size=200, min_count=1)
model_2.build_vocab(word_list)
total_examples = model_2.corpus_count


In [48]:
model_2.build_vocab([list(model.wv.vocab.keys())], update=True)
model_2.intersect_word2vec_format("/content/gdrive/MyDrive/ko.bin.gz", binary=False)

In [50]:
model_2.train(word_list, total_examples=total_examples, epochs=50)

(6064084, 6327300)

In [90]:
model_2.most_similar('양말')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('신발', 0.772346556186676),
 ('속옷', 0.7397254109382629),
 ('옷감', 0.7003517150878906),
 ('반바지', 0.6818591356277466),
 ('검은색', 0.6801407337188721),
 ('양복', 0.6758757829666138),
 ('가죽', 0.6746922731399536),
 ('남색', 0.6663075685501099),
 ('스타킹', 0.6629595756530762),
 ('셔츠', 0.659692645072937)]

In [62]:
def vectors(document_list):
    document_embedding_list = []

    # 각 문서에 대해서
    for line in document_list:
        doc2vec = None
        count = 0
        for word in line:
            if word in model_2.wv.vocab:
                count += 1
                # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
                if doc2vec is None:
                    doc2vec = model_2[word]
                else:
                    doc2vec = doc2vec + model_2[word]

        if doc2vec is not None:
            # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)

    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return document_embedding_list

In [63]:
document_embedding_list = vectors(word_list)
print('문서 벡터의 수 :',len(document_embedding_list))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


문서 벡터의 수 : 589


In [64]:
cosine_similarities = cosine_similarity(document_embedding_list, document_embedding_list)
print('코사인 유사도 매트릭스의 크기 :',cosine_similarities.shape)


코사인 유사도 매트릭스의 크기 : (589, 589)


In [65]:
def recommendations(title):
    restaurant = full_data    
    # 음식점의 인덱스를 리턴받아 idx에 저장.
    indices = pd.Series(full_data.index, index = full_data['store_name']).drop_duplicates()    
    idx = indices[title]
    print(idx)
    # 블로그 단어와 유사한 5개 선정.
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:11]
    print(sim_scores)
    # 가장 유사한 5개 인덱스
    restaurant_indices = [i[0] for i in sim_scores]

    # 전체 데이터프레임에서 해당 인덱스의 행만 추출. 5개의 행을 가진다.
    recommend = restaurant.iloc[restaurant_indices].reset_index(drop=True)

    # for index, row in recommend.iterrows():
    #     print(row[['store_name', 'store_type', 'star_point', 'star_point_qty','menu','addr']])
    
    return recommend

In [66]:
recommand = recommendations("촙촙")
recommand

433
[(511, 0.9422603), (519, 0.9332976), (307, 0.92922175), (502, 0.9228063), (420, 0.922183), (535, 0.91858155), (95, 0.91797376), (269, 0.91755104), (531, 0.91591597), (53, 0.9141951)]


,store_name,store_type,star_point,star_point_qty,menu,blog_keyword,addr
0,을지깐깐,베트남음식,4.52,658,"Bánh canh cua 게살국수\n대표,Cơm tấm 고기덮밥\n대표,Phở bò...","지인,준,을지,찬찬,원래,내,롯데,호텔,샤인,머,스캣,애프터눈티,그렇다,점심,간,곳...",서울특별시 중구 저동2가 47-10 1층
1,진작,일식당,4.45,1374,"진작 후토마끼\n대표,한우대창덮밥\n대표,진작모듬카츠,명란까르보나라,봉골레,마구로덮...","을지로,역,맛집,대창,밥,후토,마키,맛있다,곳,진작,만,주니,을지로,데이트,저번,때...",서울특별시 중구 초동 107-10번지
2,알로,"스테이크,립",4.79,235,"부채살스테이크 정식(평일점심)\n대표,토마호크스테이크 2~3인Set(750g)\n대...","회사,근처,자주,곳,중,하나,을지로,데이트,주말,데이트,사람,많다,장소,을지로,맛집...",서울특별시 중구 을지로2가 199-39번지 204205호
3,찜이로닭,찜닭,4.31,245,"찜이로닭 (소),매콤숙주 찜이로닭 (소),순살 혼닭 찜닭,매콤 숙주 혼닭","지난,주말,교외,나들이,시내,점심,집,게,좋다,것,같다,을지로,맛집,밥,먹기,예전,...",서울특별시 중구 저동2가 7-2
4,카렝,카레,4.36,47,"카레라 카렝\n대표,게살고로케카렝\n대표,돈 카렝,치킨 카렝,새우 카렝,우동 카렝,...","얼마,전,지인,힙,을지로,접선,뒷북몽,이,때,점심,조금,지난,오후,시간대,것,같다,...",서울특별시 중구 을지로3가 227번지 2층
5,을지로제,"스파게티,파스타전문",4.46,236,"누룽지 통닭 파스타(1.5인분)\n대표,킹 새우 파스타\n대표,싸가지 버섯 파스타,...","오늘,힙지로,간만,파스타,맛집,발견,기분,좋다,마음,포스팅,을지로,제,위치,을지로,...",서울특별시 중구 초동 107-5번지
6,을지로술집,요리주점,4.49,180,"숙성회모듬\n대표,등심돈까스,안심돈까스,돈까스카레덮밥,치킨카레덮밥,새우튀김카레덮밥,...","퇴근,후,혼자,잔,을지로,역,가까이,위치,있다,수월하다,입구,마자,낚싯대,통발,등,...",서울특별시 중구 주교동 43-21번지
7,가야성,중식당,4.27,406,"짜장면,우동/짬뽕/간짜장/울면,가리비 낙지짬뽕,삼선우동/삼선짬뽕/삼선울면/삼선짜장/...","커다랗다,낙지,마리,통째,가리비,낙지,짬뽕,해장,필요하다,때,짬뽕,전문점,점심,세트...",서울특별시 중구 을지로2가 199-40번지 외1필지(1층 114호 128호)
8,네평반,아시아음식,4.68,65,"가지튀김,마늘새우당면찜,청양마요새우,새우완탕면,어향동고,차돌양지배추찜,칠리새우,마파...","안녕하다,라미,오늘,일명,힙지로,을지로,언니,평반,오라,언니,집,말,알,언니네,주소...",서울특별시 중구 을지로3가 342-3번지
9,을지분식,종합분식,4.37,481,"직화 우삼겹 즉석떡볶이(2인)\n대표,소곱창 즉석떡볶이(2인)\n대표,즉석떡볶이(2...","힙지로,시작,즈음,을지로,길맥,년,다시,방문,힙지,그때,투,박하다,골목,지나,길거리...",서울특별시 중구 을지로5가 149-1번지


In [67]:
full_data['cate_mix'] = full_data['store_type'] + full_data['menu'].fillna('*')
full_data['cate_mix'] = full_data['cate_mix'].str.replace('/', ',')

In [68]:
count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
place_category = count_vect_category.fit_transform(full_data['cate_mix']) 
place_simi_cate = cosine_similarity(place_category, place_category) 
place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1]
place_simi_review_sorted_ind = cosine_similarities.argsort()[:, ::-1]


In [84]:
place_simi_co = (
                 + place_simi_cate * 0.5 # 카테고리 유사도
                 + cosine_similarities * 1 # 리뷰 텍스트 유사도                
                 + np.repeat([pd.to_numeric(full_data['star_point']).values], len(full_data['star_point']) , axis=0) * 0.0005            #  별점 가중치
                 + np.repeat([pd.to_numeric(full_data['star_point_qty']).values], len(full_data['star_point_qty']) , axis=0) * 0.00001    # 별점 평가가 얼마나 많이 됐는지
                 )
place_simi_co_sorted_ind = place_simi_co.argsort()[:, ::-1] 

In [85]:
def find_simi_place(df, sorted_ind, place_name, top_n=10):
    
    place_title = df[full_data['store_name'] == place_name]
    place_index = place_title.index.values
    similar_indexes = sorted_ind[place_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]

find_simi_place(full_data, place_simi_co_sorted_ind, '신라마라탕', 5)

,store_name,store_type,star_point,star_point_qty,menu,blog_keyword,addr,cate_mix
393,신라마라탕,중식당,4.02,68,"사골마라탕 (100G)\n대표,마라샹궈 (100G)\n대표,꿔바러우 (소),짜장면,...","매콤,다양하다,재료,마라샹궈,내,원하다,재료,하나,하나,것,나름,장점,메뉴,중,하나...",서울특별시 중구 을지로6가 17-2번지 지하2층,"중식당사골마라탕 (100G)\n대표,마라샹궈 (100G)\n대표,꿔바러우 (소),짜..."
268,신룽푸마라탕,신룽푸마라탕,4.42,1137,"마라탕 (100g 당),마라샹궈 (100g 당),마라반 (100g 당),꿔바로우,랑피","안녕하다,핸잉,용,오늘,소개,곳,명동,마르다,탕,맛집,룽,푸마,탕,이다,영업,시간,...",서울특별시 중구 을지로2가 199-76 (지상12층),"신룽푸마라탕마라탕 (100g 당),마라샹궈 (100g 당),마라반 (100g 당),..."
339,해룡마라룽샤,중식당,4.49,278,"매운가재,매운잉어구이,매운꽃게,매운새우,매운바지락,매운동태,삼겹살감자조림,탕수육,매...","해룡,마르다,룽샤,영업,시간,매일,만,정말,좋아하다,곳,해룡,마르다,룽샤,맵다,가재...",서울특별시 중구 을지로6가 18-131번지 1층,"중식당매운가재,매운잉어구이,매운꽃게,매운새우,매운바지락,매운동태,삼겹살감자조림,탕수..."
478,제주애월카츠,돈가스,4.32,200,모짜렐라 치즈 카츠\n대표,"저,지금,연속,동대문,욤,의도,건,아니다,동대문,동대문,타,맛집,소개,제주,애월,카...",서울특별시 중구 초동 17-4번지 한신빌딩,돈가스모짜렐라 치즈 카츠\n대표
220,마부마라탕&마라샹궈,중식당,4.52,170,NaN,"여러분,저,마르다,탕,처,돌이,과연,제,처,음식,있다,의아,마부,마르다,탕,동대문역...",서울특별시 중구 초동 53-6번지,중식당*


In [151]:
tokenizer = Okt()
keyword =[]
def restraunt_recommandation(sentence): 
    tokenized = tokenizer.pos(sentence, norm=True, stem=True)   
    okt_filtering = [x for x, y in tokenized if y in ['Noun', 'Adjective']]        
    # keyword.append( ','.join(okt_filtering)  ) 
    sentence_vector = None
    for word in okt_filtering:       
        if word in model_2.wv.vocab:            
            if sentence_vector is None:
                sentence_vector = model_2[word]
            else:
                sentence_vector = sentence_vector + model_2[word]
    if sentence_vector is not None:       
        sentence_vector = sentence_vector / len(okt_filtering)
        cs = cosine_similarity(document_embedding_list, [sentence_vector])
        result = find_simi_place(full_data, place_simi_co_sorted_ind, full_data.iloc[cs.argmax()]['store_name'], 5)
        return result      
        # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        #     print(result[['store_name', 'store_type', 'star_point', 'star_point_qty', 'menu', 'addr']])
        
    else:
        print('찾는 결과가 없습니다. 다른 키워드로 검색하세요.')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


,store_name,store_type,star_point,star_point_qty,menu,blog_keyword,addr,cate_mix
587,다크문,바(BAR),4.21,18,"아메리카노,칵테일","을지로,칵테일,사이키델릭,바,다크,문,요즘,영업,시간,제한,있다,그렇다,예전,차,차...",서울특별시 중구 저동2가 69-1번지,"바(BAR)아메리카노,칵테일"
440,을지로 브이,바(BAR),4.57,102,NaN,"을지로,브이,요거트,장사,옴청,블로그,후기,없다,저,주로,테이크아웃,모,예쁘다,저,...",서울특별시 중구 을지로3가 349-1번지 3층,바(BAR)*
561,선셋 레코드,카페,4.51,293,"커피,칵테일","을지로,힙,칵테일,바,아쉽다,선셋,레코드,만,을지로,예,고민,고민,선셋,레코드,처음...",서울특별시 중구 산림동 207-2번지 대림상가 356호,"카페커피,칵테일"
442,을지로맨틱,바(BAR),4.63,49,"와인,싱글몰트 위스키,칵테일,치즈 플레이트,햄&치즈 플레이트,몰트 플레이트,칩 코코...","지난,주,퇴근,시간,친구,회사,근처,을지로,을지로,와인,저,몇,군데,을지로,와인,친...",서울특별시 중구 을지로6가 17-2번지 지하2층,"바(BAR)와인,싱글몰트 위스키,칵테일,치즈 플레이트,햄&치즈 플레이트,몰트 플레이..."
507,빈티지라이언,바(BAR),4.35,97,"코젤다크 생맥주\n대표,칵테일\n대표,아메리카노,우롱 밀크티,레드락 생맥주,상그리아...","혹시,세운상가,지금,젊다,친구,있다,저,어린시절,용산,전자,상가,더,핫,곳,초등학교...",서울특별시 중구 을지로4가 77번지 1층,"바(BAR)코젤다크 생맥주\n대표,칵테일\n대표,아메리카노,우롱 밀크티,레드락 생맥..."


## 키워드 기반 음식점 추천 시스템

In [158]:
result = restraunt_recommandation('부장님과 회식')
result

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


,store_name,store_type,star_point,star_point_qty,menu,blog_keyword,addr,cate_mix
83,종로생삼겹살,돼지고기구이,4.74,21,"양평해장국,내장탕,곱창전골,쌈밥,참치찌개","을지로,맛집,종로,생,삼겹살,양평,해장국,아침식사,회식,장소,추천,한국,두번째,개강...",서울특별시 중구 주교동 66-0번지,"돼지고기구이양평해장국,내장탕,곱창전골,쌈밥,참치찌개"
42,양평해장국,해장국,4.27,208,"양평해장국,내장탕,곱창전골,쌈밥,냉면","양평,해장국,전화,주소,서울,중구,수표로,안녕하다,아리,오늘,동기,야근,인하다,몸,...",서울특별시 중구 초동 19-9번지,"해장국양평해장국,내장탕,곱창전골,쌈밥,냉면"
214,우정집,오리요리,4.35,204,"생대구탕,내장탕, 대구탕 각,훈제수육","충무로,위치,대구,탕,전문점,우정,집,영덕,회식,차로,곳,우정,집,이다,결과,차,요...",서울특별시 중구 을지로6가 18-84번지 외1필지(미진빌딩 지상4층),"오리요리생대구탕,내장탕, 대구탕 각,훈제수육"
45,최가네황소곱창,"곱창,막창,양",4.43,288,"황소곱창 구이 1인분\n대표,곱창전골 大\n대표,곱창전골 中,곱창전골 小,우렁된장찌...","안녕하다,지난주,훈트씨,인생,이제껏,한번,경험,못,곱창,맛,진짜,인생,곱창,집,군데...",서울특별시 중구 을지로5가 79-1번지,"곱창,막창,양황소곱창 구이 1인분\n대표,곱창전골 大\n대표,곱창전골 中,곱창전골 ..."
99,청와옥 을지로3가직영점,"순대,순댓국",4.43,2858,"순대국밥,편백정식,동해 오징어숯불구이,찹쌀순대,모둠순대,수육,육회 소,육사시미,얼큰...","전,병원,입원,때,엄마,을지로,역,맛집,포장,준,있다,그,때,진짜,맛있다,매장,가면...",서울특별시 중구 을지로2가 199-53 (지상1층),"순대,순댓국순대국밥,편백정식,동해 오징어숯불구이,찹쌀순대,모둠순대,수육,육회 소,육..."
